In [1]:
# Import objects from pyomo package
from pyomo.environ import (ConcreteModel,
                           SolverFactory,
                           Constraint,
                           Expression,
                           TransformationFactory,
                           value,
                           units as pyunits)

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

# Import idaes logger to set output levels
import idaes.logger as idaeslog

from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

# Todo: import Flash unit model from idaes.generic_models.unit_models
from idaes.generic_models.unit_models import Flash

In [2]:
from idaes.core.util.model_statistics import degrees_of_freedom, large_residuals_set

import pyomo.contrib.parmest.parmest as parmest

In [3]:
# Import plotting functions
import matplotlib.pyplot as plt

# Import numpy library 
import numpy as np

# Import Pandas
import pandas as pd

In [4]:
from R125_R32_PR3 import configuration

# Load data from csv
data = pd.read_csv('R32R125298.csv')

In [5]:
def PR_model(data):
    
    m = ConcreteModel()

    m.fs = FlowsheetBlock(default={"dynamic": False})

    m.fs.properties = GenericParameterBlock(default=configuration)

    m.fs.state_block = m.fs.properties.state_block_class(
        default={"parameters": m.fs.properties,
                 "defined_state": True})

    m.fs.state_block.flow_mol.fix(1)
    m.fs.state_block.temperature.fix(298.15)
    m.fs.state_block.pressure.fix(1669700)
    m.fs.state_block.mole_frac_comp["R32"].fix(0.2688)
    m.fs.state_block.mole_frac_comp["R125"].fix(0.7312)
    
    m.fs.properties.PR_kappa['R32', 'R32'].fix(0)
    m.fs.properties.PR_kappa['R125', 'R125'].fix(0)
    m.fs.properties.PR_kappa['R32', 'R125'].fix(0.004)
    m.fs.properties.PR_kappa['R125', 'R32'].fix(0.004)

    # Initialize the flash unit
    m.fs.state_block.initialize(outlvl=idaeslog.CRITICAL)

    # Fix at actual temperature
    m.fs.state_block.pressure.unfix()
    m.fs.state_block.temperature.fix(float(data["temperature"]))
#     m.fs.state_block.pressure.fix(float(data["pressure"]))
#     m.fs.state_block.mole_frac_comp["R32"].fix(float(data["x_R32"]))
    m.fs.state_block.mole_frac_comp["R125"].unfix()
    m.fs.state_block.mole_frac_comp["R32"].unfix()
#     m.fs.state_block.mole_frac_comp["R32"].fix(float(data["x_R32"]))
    m.fs.state_block.mole_frac_phase_comp['Liq', 'R125'].fix(float(data["x_R125"]))
    m.fs.state_block.mole_frac_phase_comp['Liq', 'R32'].fix(float(data["x_R32"]))
#     m.fs.state_block.mole_frac_phase_comp['Liq', 'R32'].unfix()
#     m.fs.state_block.mole_frac_phase_comp['Vap', 'R125'].fix(float(data["y_R125"]))
#     m.fs.state_block.mole_frac_phase_comp['Vap', 'R32'].fix(float(data["y_R32"]))
#     m.fs.state_block.mole_frac_phase_comp['Vap', 'R32'].unfix()
#     m.fs.state_block.mole_frac_phase_comp['Vap', 'R125'].unfix()
#     m.fs.state_block.mole_frac_phase_comp['Vap', 'R125'].unfix()
    
    m.fs.kappas = Constraint(expr=m.fs.properties.PR_kappa['R32', 'R125'] == 
                    m.fs.properties.PR_kappa['R125', 'R32'])
#     m.fs.properties.PR_kappa['R32', 'R125'] = Expression(expr=value(m.fs.properties.PR_kappa['R125', 'R32']))
    
    # Set bounds on variables to be estimated
    m.fs.properties.PR_kappa['R32', 'R125'].setlb(-3)
    m.fs.properties.PR_kappa['R32', 'R125'].setub(3)

    m.fs.properties.PR_kappa['R125', 'R32'].setlb(-3)
    m.fs.properties.PR_kappa['R125', 'R32'].setub(3)

    # Return initialized flash model
    return m

In [6]:
import pytest

test_data = {"temperature": 298.15, "pressure": 1669700, "x_R32":0.7312, 
             "x_R125":0.2688, "y_R125":0.2501, "y_R32":0.7499}

m = PR_model(test_data)

DOF_initial = degrees_of_freedom(m)
print("The initial DOF is {0}".format(DOF_initial))

# Check that degrees of freedom is 0
# assert degrees_of_freedom(m) == 0

2021-03-13 19:20:58 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
The initial DOF is 0


In [7]:
variable_name = ["fs.properties.PR_kappa['R32', 'R125']",
                 "fs.properties.PR_kappa['R125', 'R32']"]

In [8]:
def SSE(m, data):
    expr = ((float(data["pressure"]) - m.fs.state_block.pressure)**2)
    return expr*1e-7

# def SSE(m, data):
#     expr = (
#             (float(data["y_R125"]) -
#              m.fs.state_block.mole_frac_phase_comp["Vap", "R125"])**2
# #             + (float(data["x_R125"]) -
# #              m.fs.state_block.mole_frac_phase_comp["Liq", "R125"])**2
#            )
    
#     return expr*1e4


In [9]:
solver_options = {'tol': 1e-4}
pest = parmest.Estimator(PR_model, data, variable_name, SSE, solver_options)
# solver_options = {'tol': 1e-6, 'max_iter':100}

obj_value, parameters = pest.theta_est()

2021-03-13 19:20:59 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties phase Liq was not assigned a type. Using generic Phase object.
2021-03-13 19:20:59 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties phase Vap was not assigned a type. Using generic Phase object.
2021-03-13 19:20:59 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties component R32 was not assigned a type. Using generic Component object.
2021-03-13 19:20:59 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties component R125 was not assigned a type. Using generic Component object.
2021-03-13 19:21:05 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-03-13 19:21:05 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties phase Liq was not assigned a type. Using generic Phase object.
2021-03-13 1

2021-03-13 19:21:42 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties phase Vap was not assigned a type. Using generic Phase object.
2021-03-13 19:21:42 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties component R32 was not assigned a type. Using generic Component object.
2021-03-13 19:21:42 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties component R125 was not assigned a type. Using generic Component object.
2021-03-13 19:21:46 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-03-13 19:21:46 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties phase Liq was not assigned a type. Using generic Phase object.
2021-03-13 19:21:46 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties phase Vap was not assigned a type. Using generic Phase object.
2021-03-13 1

2021-03-13 19:22:28 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties component R32 was not assigned a type. Using generic Component object.
2021-03-13 19:22:28 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties component R125 was not assigned a type. Using generic Component object.
2021-03-13 19:22:31 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-03-13 19:22:31 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties phase Liq was not assigned a type. Using generic Phase object.
2021-03-13 19:22:31 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties phase Vap was not assigned a type. Using generic Phase object.
2021-03-13 19:22:31 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties component R32 was not assigned a type. Using generic Component object.
2021

2021-03-13 19:23:15 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties component R125 was not assigned a type. Using generic Component object.
2021-03-13 19:23:21 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-03-13 19:23:21 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties phase Liq was not assigned a type. Using generic Phase object.
2021-03-13 19:23:21 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties phase Vap was not assigned a type. Using generic Phase object.
2021-03-13 19:23:21 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties component R32 was not assigned a type. Using generic Component object.
2021-03-13 19:23:21 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties component R125 was not assigned a type. Using generic Component object.
202

2021-03-13 19:24:08 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-03-13 19:24:08 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties phase Liq was not assigned a type. Using generic Phase object.
2021-03-13 19:24:08 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties phase Vap was not assigned a type. Using generic Phase object.
2021-03-13 19:24:08 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties component R32 was not assigned a type. Using generic Component object.
2021-03-13 19:24:08 [WARNING] idaes.generic_models.properties.core.generic.generic_property: fs.properties component R125 was not assigned a type. Using generic Component object.
2021-03-13 19:24:11 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-03-13 19:24:11 [WARNING] idaes.generic_models.properties

...

2585  1.2375983e-01 3.17e-06 2.08e+01  -2.5 2.16e+03  -7.9 1.00e+00 1.56e-02h  7
2586  1.2757984e-01 3.15e-06 1.40e+01  -2.5 3.75e+02  -7.5 1.00e+00 1.00e+00H  1
2587  1.3016141e-01 3.14e-06 1.08e+01  -2.5 2.39e+02  -7.0 1.00e+00 1.00e+00H  1
2588  1.3147428e-01 3.13e-06 3.17e+00  -2.5 1.23e+02  -6.6 1.00e+00 1.00e+00h  1
2589  1.4053050e-01 3.11e-06 1.13e+02  -2.5 7.70e+02  -7.1 1.00e+00 1.00e+00H  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2590  1.4072594e-01 3.11e-06 3.16e+02  -2.5 7.95e+03  -6.7 7.45e-01 1.95e-03h 10
2591  1.4096533e-01 3.11e-06 1.08e+02  -2.5 3.00e+02  -6.2 1.00e+00 6.25e-02h  5
2592  1.4207306e-01 3.11e-06 1.16e+01  -2.5 8.61e+01  -5.8 1.00e+00 1.00e+00H  1
2593  1.4218809e-01 3.11e-06 3.89e+01  -2.5 5.65e+02  -6.3 1.00e+00 1.56e-02h  7
2594  1.4380112e-01 3.10e-06 2.84e+01  -2.5 1.24e+02  -5.9 1.00e+00 1.00e+00H  1
2595  1.4413273e-01 3.10e-06 2.58e+02  -2.5 3.18e+03  -6.3 1.00e+00 7.81e-03h  8
2596  1.4728687e-01 5.31e-05

2688r 3.5194689e-01 1.42e-03 3.42e+02  -8.6 7.62e+02  -8.9 1.00e+00 3.03e-01h  1
2689r 3.7757017e-01 1.34e-03 3.25e+02  -8.6 2.46e+04  -9.3 7.72e-02 5.90e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2690r 3.7493938e-01 8.33e-04 1.13e+02  -8.6 2.19e+02    -  8.26e-05 4.02e-01f  1
2691r 3.7487397e-01 8.13e-04 4.81e+02  -8.6 7.20e+01  -8.0 8.45e-01 2.43e-02f  1
2692r 3.7486374e-01 8.08e-04 5.23e+02  -8.6 6.12e+01  -7.6 1.56e-01 6.04e-03h  1
2693r 3.7468629e-01 8.05e-04 5.23e+02  -8.6 2.68e+03    -  7.20e-04 3.33e-03h  1
2694r 3.7442756e-01 7.96e-04 5.17e+02  -8.6 1.16e+03  -8.1 4.46e-02 1.08e-02h  1
2695r 3.7443090e-01 7.96e-04 4.87e+02  -8.6 2.39e+03  -8.5 1.00e+00 7.33e-05h  1
2696r 3.7324187e-01 5.48e-04 2.81e+02  -8.6 1.23e+02    -  1.00e+00 3.18e-01f  1
2697r 3.7355724e-01 1.48e-05 5.12e-01  -8.6 5.51e+01    -  1.00e+00 1.00e+00f  1
2698r 3.7696564e-01 1.13e-04 2.09e+01  -8.6 2.48e+02    -  1.00e+00 6.55e-01h  1
2699r 3.7724265e-01 8.32e-05

2785r 3.7820763e-01 8.25e-05 4.12e+02  -8.6 8.59e+01    -  1.00e+00 6.05e-02h  1
2786r 3.7964545e-01 4.06e-05 9.52e-02  -8.6 7.93e+01    -  1.00e+00 1.00e+00f  1
2787r 3.7974514e-01 7.74e-07 7.18e-03  -8.6 9.66e+00    -  1.00e+00 1.00e+00h  1
2788  3.7782700e-01 1.08e-05 2.93e+02  -2.5 6.43e+03    -  9.99e-01 3.91e-03f  9
2789  1.2158464e-01 3.58e-02 2.24e+03  -2.5 6.35e+03    -  1.00e+00 8.78e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2790  2.1046689e-01 1.72e-03 2.90e+02  -2.5 3.17e+03    -  1.00e+00 1.00e+00h  1
2791  1.9798784e-01 8.67e-04 1.46e+02  -2.5 4.49e+02  -7.8 1.00e+00 4.96e-01h  2
2792  1.8329640e-01 6.60e-04 1.13e+02  -2.5 1.70e+03  -8.3 1.00e+00 2.28e-01h  2
2793  1.3707987e-01 4.57e-04 7.51e+01  -2.5 3.29e+03    -  1.00e+00 3.35e-01h  2
2794  1.1957335e-01 2.34e-04 3.73e+01  -2.5 1.67e+03    -  1.00e+00 5.00e-01h  2
2795  1.1402943e-01 1.19e-04 1.82e+01  -2.5 8.60e+02    -  1.00e+00 5.00e-01h  2
2796  1.1067217e-01 1.23e-05

2879  1.0923815e-01 3.32e-05 1.92e+01  -2.5 1.27e+02    -  1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2880  1.1004283e-01 1.61e-05 1.55e+01  -2.5 1.45e+02    -  1.00e+00 1.00e+00h  1
2881  1.1087496e-01 3.72e-06 1.26e+01  -2.5 1.61e+02    -  1.00e+00 1.00e+00h  1
2882  1.1182311e-01 3.66e-06 1.09e+01  -2.5 1.89e+02    -  1.00e+00 1.00e+00h  1
2883  1.1303417e-01 3.58e-06 9.23e+00  -2.5 2.35e+02    -  1.00e+00 1.00e+00h  1
2884  1.1482890e-01 3.43e-06 7.56e+00  -2.5 3.30e+02    -  1.00e+00 1.00e+00h  1
2885  1.1670738e-01 3.26e-06 5.16e+00  -2.5 6.82e+02    -  1.00e+00 5.00e-01h  2
2886  1.1704630e-01 3.25e-06 2.30e+01  -2.5 4.15e+03    -  1.00e+00 1.56e-02h  7
2887  1.1724304e-01 3.25e-06 1.10e+02  -2.5 1.92e+04    -  1.00e+00 1.95e-03h 10
2888  1.1729119e-01 3.42e-06 5.37e+00  -2.5 6.57e+00  -6.9 1.00e+00 1.00e+00h  1
2889  1.1755284e-01 3.39e-06 5.12e+00  -2.5 3.47e+01  -7.3 1.00e+00 1.00e+00h  1
iter    objective    inf_pr 

2971  3.7515912e-01 1.72e-05 2.92e+02  -2.5 6.34e+03    -  9.42e-03 1.22e-04f 14
2972  1.1030729e-01 4.90e-02 7.41e+02  -2.5 6.34e+03    -  9.67e-03 1.00e+00f  1
2973  1.0570561e-01 5.17e-04 2.08e+02  -2.5 1.44e+03    -  1.00e+00 1.00e+00h  1
2974  1.0723777e-01 1.18e-04 1.43e+02  -2.5 1.53e+02    -  1.00e+00 1.00e+00h  1
2975  1.0822624e-01 2.13e-04 1.17e+02  -2.5 1.37e+02    -  1.00e+00 1.00e+00h  1
2976  1.0902664e-01 1.15e-04 6.71e+01  -2.5 9.25e+01    -  1.00e+00 1.00e+00h  1
2977  1.0979446e-01 4.70e-05 5.97e+01  -2.5 1.43e+02    -  1.00e+00 1.00e+00h  1
2978  1.1060010e-01 2.05e-05 1.57e+01  -2.5 1.57e+02    -  1.00e+00 1.00e+00h  1
2979  1.1154909e-01 3.66e-06 1.24e+01  -2.5 1.90e+02    -  1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2980  1.1277615e-01 3.57e-06 9.17e+00  -2.5 2.37e+02    -  1.00e+00 1.00e+00h  1
2981  1.1460712e-01 3.42e-06 7.51e+00  -2.5 3.35e+02    -  1.00e+00 1.00e+00h  1
2982  1.1657510e-01 3.25e-06

In [10]:
print("The SSE at the optimal solution is %0.6f" % obj_value)
print()
print("The values for the parameters are as follows:")
for k,v in parameters.items():
    print(k, "=", v)

The SSE at the optimal solution is 0.146662

The values for the parameters are as follows:
fs.properties.PR_kappa[R125,R32] = -0.002534978593055062
fs.properties.PR_kappa[R32,R125] = -0.002534978592607153


In [11]:
pest.ef_instance.display()

Model MASTER

  Variables:
    MASTER_BLEND_VAR_RootNode : Size=2, Index=MASTER_BLEND_VAR_RootNode_index
        Key                              : Lower : Value                  : Upper : Fixed : Stale : Domain
        fs.properties.PR_kappa[R125,R32] :  None : -0.0025349785930550623 :  None : False : False :  Reals
        fs.properties.PR_kappa[R32,R125] :  None :  -0.002534978592607153 :  None : False : False :  Reals

  Objectives:
    MASTER : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 0.1466619582921262

  Constraints:
    MASTER_BLEND_CONSTRAINT_RootNode : Size=132
        Key : Lower : Body                    : Upper
          1 :   0.0 :   -2.93123038862253e-11 :   0.0
          2 :   0.0 : -1.6622289482348673e-12 :   0.0
          3 :   0.0 :  -2.836401686440171e-12 :   0.0
          4 :   0.0 :  2.3735388654522183e-12 :   0.0
          5 :   0.0 :  2.6060685727469224e-12 :   0.0
          6 :   0.0 :   3.081135173388372e-12 :   0.0

                    Key                    : Lower : Body                   : Upper
                    ('Vap', 'Liq', 'R125') :   0.0 :  1.451192843671123e-07 :   0.0
                     ('Vap', 'Liq', 'R32') :   0.0 : 2.0235546571711893e-09 :   0.0
    Block Experiment1
    
      Variables:
        None
    
      Objectives:
        None
    
      Constraints:
        None
    
      Blocks:
        Block Experiment1.fs
        
          Variables:
            None
        
          Objectives:
            None
        
          Constraints:
            kappas : Size=1
                Key  : Lower : Body                   : Upper
                None :   0.0 : 4.4539502294660416e-13 :   0.0
        
          Blocks:
            Block Experiment1.fs.properties
            
              Variables:
                PR_kappa : PR binary interaction parameters
                    Size=4, Index=Experiment1.fs.properties.PR_kappa_index
                    Key              : Lower : 

                    Key                    : Lower : Body                   : Upper
                    ('Vap', 'Liq', 'R125') :   0.0 :  1.845179575798994e-08 :   0.0
                     ('Vap', 'Liq', 'R32') :   0.0 : 2.5437283390772336e-09 :   0.0
                eq_mole_frac_tbub : Size=1
                    Key            : Lower  : Body               : Upper
                    ('Vap', 'Liq') : 1000.0 : 1000.0000000020997 : 1000.0
                _teq_constraint_Vap_Liq : Size=1
                    Key  : Lower : Body                  : Upper
                    None :   0.0 : 6.020854925736785e-10 :   0.0
                eq_temperature_dew : Size=2
                    Key                    : Lower : Body                   : Upper
                    ('Vap', 'Liq', 'R125') :   0.0 : 2.3203661214665772e-14 :   0.0
                     ('Vap', 'Liq', 'R32') :   0.0 : -2.424727085781342e-13 :   0.0
                eq_mole_frac_tdew : Size=1
                    Key            : Low

                    ('Vap', 'Liq', 'R125') :   0.0 : 1.4155343563970746e-14 :   0.0
                     ('Vap', 'Liq', 'R32') :   0.0 : -1.341149413747189e-13 :   0.0
                eq_mole_frac_tdew : Size=1
                    Key            : Lower  : Body   : Upper
                    ('Vap', 'Liq') : 1000.0 : 1000.0 : 1000.0
                equilibrium_constraint : Size=2
                    Key                    : Lower : Body                  : Upper
                    ('Vap', 'Liq', 'R125') :   0.0 : 2.354808148474774e-08 :   0.0
                     ('Vap', 'Liq', 'R32') :   0.0 :  6.39050602713187e-09 :   0.0
    Block Experiment3
    
      Variables:
        None
    
      Objectives:
        None
    
      Constraints:
        None
    
      Blocks:
        Block Experiment3.fs
        
          Variables:
            None
        
          Objectives:
            None
        
          Constraints:
            kappas : Size=1
                Key  : Lower : Body 

                    ('Vap', 'Liq', 'R125') :   0.0 :  8.104628079763643e-15 :   0.0
                     ('Vap', 'Liq', 'R32') :   0.0 : 2.6645352591003757e-15 :   0.0
                eq_mole_frac_tbub : Size=1
                    Key            : Lower  : Body   : Upper
                    ('Vap', 'Liq') : 1000.0 : 1000.0 : 1000.0
                _teq_constraint_Vap_Liq : Size=1
                    Key  : Lower : Body                   : Upper
                    None :   0.0 : -5.684341886080802e-14 :   0.0
                eq_temperature_dew : Size=2
                    Key                    : Lower : Body                   : Upper
                    ('Vap', 'Liq', 'R125') :   0.0 :  5.662137425588298e-15 :   0.0
                     ('Vap', 'Liq', 'R32') :   0.0 : 1.0880185641326534e-14 :   0.0
                eq_mole_frac_tdew : Size=1
                    Key            : Lower  : Body   : Upper
                    ('Vap', 'Liq') : 1000.0 : 1000.0 : 1000.0
                equilib

...

                eq_mole_frac_tbub : Size=1
                    Key            : Lower  : Body               : Upper
                    ('Vap', 'Liq') : 1000.0 : 1000.0000000078097 : 1000.0
                _teq_constraint_Vap_Liq : Size=1
                    Key  : Lower : Body                   : Upper
                    None :   0.0 : 1.3022827261011116e-09 :   0.0
                eq_temperature_dew : Size=2
                    Key                    : Lower : Body                   : Upper
                    ('Vap', 'Liq', 'R125') :   0.0 : 2.7755575615628914e-15 :   0.0
                     ('Vap', 'Liq', 'R32') :   0.0 : -3.952393967665557e-14 :   0.0
                eq_mole_frac_tdew : Size=1
                    Key            : Lower  : Body   : Upper
                    ('Vap', 'Liq') : 1000.0 : 1000.0 : 1000.0
                equilibrium_constraint : Size=2
                    Key                    : Lower : Body                  : Upper
                    ('Vap', 'Liq', '

              Blocks:
                Block Experiment61.fs.properties.Liq
                
                  Variables:
                    None
                
                  Objectives:
                    None
                
                  Constraints:
                    None
                Block Experiment61.fs.properties.Vap
                
                  Variables:
                    None
                
                  Objectives:
                    None
                
                  Constraints:
                    None
                Block Experiment61.fs.properties.R32
                
                  Variables:
                    pressure_crit : Size=1, Index=None
                        Key  : Lower : Value     : Upper : Fixed : Stale : Domain
                        None :  None : 5830000.0 :  None :  True :  True :  Reals
                    temperature_crit : Size=1, Index=None
                        Key  : Lower : Value : Upper : Fixed : S

                _mole_frac_tbub : Vapor mole fractions at bubble temperature
                    Size=2, Index=Experiment62.fs.state_block._mole_frac_tbub_index
                    Key                    : Lower : Value               : Upper : Fixed : Stale : Domain
                    ('Vap', 'Liq', 'R125') :     0 : 0.28560078885684165 :  None : False : False :  Reals
                     ('Vap', 'Liq', 'R32') :     0 :  0.7143992111431583 :  None : False : False :  Reals
                temperature_dew : Dew point temperature
                    Size=1, Index=Experiment62.fs.properties._pe_pairs
                    Key            : Lower : Value             : Upper : Fixed : Stale : Domain
                    ('Vap', 'Liq') :    10 : 303.2707054881591 :  1000 : False : False :  Reals
                _mole_frac_tdew : Liquid mole fractions at dew temperature
                    Size=2, Index=Experiment62.fs.state_block._mole_frac_tdew_index
                    Key                    

                        None :  None : 0.3278 :  None :  True :  True :  Reals
                    cp_mol_ig_comp_coeff_D : Parameter D for ideal gas molar heat capacity
                        Size=1, Index=None
                        Key  : Lower : Value     : Upper : Fixed : Stale : Domain
                        None :  None : -0.000524 :  None :  True :  True :  Reals
                    enth_mol_form_vap_comp_ref : Vapor phase molar heat of formation @ Tref
                        Size=1, Index=None
                        Key  : Lower : Value   : Upper : Fixed : Stale : Domain
                        None :  None : 21200.0 :  None :  True :  True :  Reals
                    entr_mol_form_vap_comp_ref : Vapor phase molar entropy of formation @ Tref
                        Size=1, Index=None
                        Key  : Lower : Value  : Upper : Fixed : Stale : Domain
                        None :  None : -269.0 :  None :  True :  True :  Reals
                    pressure_sat

                    Key                    : Lower : Body                   : Upper
                    ('Vap', 'Liq', 'R125') :   0.0 :  1.021405182655144e-14 :   0.0
                     ('Vap', 'Liq', 'R32') :   0.0 : -1.266209359584991e-13 :   0.0
                eq_mole_frac_tbub : Size=1
                    Key            : Lower  : Body   : Upper
                    ('Vap', 'Liq') : 1000.0 : 1000.0 : 1000.0
                _teq_constraint_Vap_Liq : Size=1
                    Key  : Lower : Body                  : Upper
                    None :   0.0 : -3.77849573851563e-09 :   0.0
                eq_temperature_dew : Size=2
                    Key                    : Lower : Body                    : Upper
                    ('Vap', 'Liq', 'R125') :   0.0 :  -4.454890101257547e-08 :   0.0
                     ('Vap', 'Liq', 'R32') :   0.0 : -1.6880768055616002e-07 :   0.0
                eq_mole_frac_tdew : Size=1
                    Key            : Lower  : Body           

In [12]:
print(large_residuals_set(pest.ef_instance))

ComponentSet([])
